In [14]:
from metro import multiplex as mx
from metro import io            # for reading multiplex
from metro import utility       # for manipulating multiplex
from metro import analysis      # analytical functions
from metro import viz           # for bubble_plot()
from metro import assignment    # for reading and manipulating OD data
from metro import ita
import networkx as nx           # assigning attributes to multiplex
import pandas as pd
from copy import deepcopy

In [12]:
betas = [100, 1, 0.5, .01]

In [3]:
multi = mx.read_multi(nodes_file_name = '2_multiplex/mx_nodes.txt', 
                      edges_file_name = '2_multiplex/mx_edges.txt')

In [4]:
multi.read_od(layer = 'taz', # keys are in this layer
          key = 'taz', # this is the key attribute
          od_file = '1_data/taz_od/0_1.txt', # here's where the file lives
          sep = ' ') # this is what separates entries


In [7]:
# Rescale base costs 
dists = nx.get_edge_attributes(multi.G, 'dist_km')
times = nx.get_edge_attributes(multi.G, 'free_flow_time_m')

speed_km_m = {key : dists[key]/times[key] for key in dists}

nx.set_edge_attributes(multi.G, 'speed_km_m', speed_km_m)

street_speed = multi.mean_edge_attr_per(layers = ['streets'], 
                                    attr = 'speed_km_m', 
                                    weight_attr = 'dist_km')

metro_speed = multi.mean_edge_attr_per(layers = ['metro'], 
                                   attr = 'speed_km_m', 
                                   weight_attr = 'dist_km') 

scale = street_speed / metro_speed

multi.scale_edge_attribute(layer = 'metro', 
                       attribute = 'free_flow_time_m', 
                       beta = 1/scale)

In [16]:
con_map = {int(multi.G.node[n]['con_name']) : n for n in multi.G.node if multi.G.node[n]['layer'] == 'taz'}

In [10]:
def od_total(od):
    return np.sum(np.sum(od[o].values()) for o in od)

def simulate(beta, n):
    df = pd.read_csv('3_throughput/route_info_' + str(beta) + '.csv')
    df['congestion_impact'] = df['gradient'] * df['flow']
    df = df.sort_values('congestion_impact', ascending = False)

    sub = df.head(n)
    ratio = 1 - sub.flow.sum() / df.flow.sum() # % of flow we are targeting here

    od = deepcopy(multi.od)
    
    # Targeted removal 
    for o, d in zip(sub.o_con, sub.d_con):
        multi.od[con_map[o]][con_map[d]] = 0
    
    df = multi.run_ita(n_nodes = None, 
                    summary = True, # change this to get route tables 
                    attrname = 'congested_time_m_TEST',
                    flow_name = 'flow_TEST',
                    P = [.2, .2, .2, .2, .1, .1],
                    scale = .25)
    
    df.to_csv('3_throughput/targeted_' + str(beta) + '.csv')
    
    
    # Reset the OD
    multi.od = od
    
    # uniform removal
    for o in multi.od:
        for d in multi.od[o]:
            multi.od[o][d] *= ratio
    
    df = multi.run_ita(n_nodes = None, 
                    summary = True, # change this to get route tables 
                    attrname = 'congested_time_m_RAND',
                    flow_name = 'flow_RAND',
                    P = [.2, .2, .2, .2, .1, .1],
                    scale = .25)
    
    df.to_csv('3_throughput/uniform_' + str(beta) + '.csv')
    
    # Reset the OD again, so we can clean up. 
    multi.od = od
    
    return targeted, uniform

In [17]:
for beta in betas:
    multi.scale_edge_attribute(layer = 'metro',
                                 attribute = 'free_flow_time_m',
                                 beta = beta)
    
    targeted, uniform = simulate(beta = beta, n = 50000)
    
    multi.scale_edge_attribute(layer = 'metro',
                                 attribute = 'free_flow_time_m',
                                 beta = 1.0/beta)

KeyboardInterrupt: 